# Pendulum Swing Up

In [1]:
from copy import copy

import matplotlib.pyplot as plt
import mpld3
import numpy as np
from IPython.display import display
from pydrake.all import (
    DiagramBuilder,
    LeafSystem,
    Linearize,
    LinearQuadraticRegulator,
    MeshcatVisualizer,
    Saturation,
    SceneGraph,
    Simulator,
    StartMeshcat,
    VectorLogSink,
    wrap_to,
)
from pydrake.examples import PendulumGeometry, PendulumParams, PendulumPlant

from underactuated import running_as_notebook

if running_as_notebook:
    mpld3.enable_notebook()

In [2]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://711f3004-0a9b-43a3-abc8-ecd3b79bb722.deepnoteproject.com/7000/


In [3]:
class EnergyShapingController(LeafSystem):
    def __init__(self, pendulum):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort("state", 2)
        self.DeclareVectorOutputPort("control", 1, self.DoCalcOutput)
        self.pendulum = pendulum
        self.pendulum_context = pendulum.CreateDefaultContext()
        self.SetPendulumParams(PendulumParams())

    def SetPendulumParams(self, params):
        self.pendulum_context.get_mutable_numeric_parameter(0).SetFromVector(
            params.CopyToVector()
        )
        self.pendulum_context.SetContinuousState([np.pi, 0])
        self.desired_energy = self.pendulum.EvalPotentialEnergy(self.pendulum_context)

    def DoCalcOutput(self, context, output):
        pendulum_state = self.get_input_port(0).Eval(context)
        self.pendulum_context.SetContinuousState(pendulum_state)
        params = self.pendulum_context.get_numeric_parameter(0)
        thetadot = pendulum_state[1]
        total_energy = self.pendulum.EvalPotentialEnergy(
            self.pendulum_context
        ) + self.pendulum.EvalKineticEnergy(self.pendulum_context)
        output.SetAtIndex(
            0,
            params.damping() * thetadot
            - 0.1 * thetadot * (total_energy - self.desired_energy),
        )

In [4]:
def BalancingLQR(pendulum):
    context = pendulum.CreateDefaultContext()

    pendulum.get_input_port(0).FixValue(context, [0])
    context.SetContinuousState([np.pi, 0])

    Q = np.diag((10.0, 1.0))
    R = [1]

    linearized_pendulum = Linearize(pendulum, context)
    (K, S) = LinearQuadraticRegulator(
        linearized_pendulum.A(), linearized_pendulum.B(), Q, R
    )
    return (K, S)

In [5]:
class SwingUpAndBalanceController(LeafSystem):
    def __init__(self, pendulum):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort("state", 2)
        self.DeclareVectorOutputPort("control", 1, self.DoCalcOutput)
        (self.K, self.S) = BalancingLQR(pendulum)
        self.energy_shaping = EnergyShapingController(pendulum)
        self.energy_shaping_context = self.energy_shaping.CreateDefaultContext()

    def DoCalcOutput(self, context, output):
        pendulum_state = self.get_input_port(0).Eval(context)
        xbar = copy(pendulum_state)
        xbar[0] = wrap_to(xbar[0], 0, 2.0 * np.pi) - np.pi

        # If x'Sx <= 2, then use the LQR controller
        if xbar.dot(self.S.dot(xbar)) < 2.0:
            output.SetFromVector(-self.K.dot(xbar))
        else:
            self.energy_shaping.get_input_port(0).FixValue(
                self.energy_shaping_context, pendulum_state
            )
            output.SetFromVector(
                self.energy_shaping.get_output_port(0).Eval(self.energy_shaping_context)
            )

In [6]:
builder = DiagramBuilder()

pendulum = builder.AddSystem(PendulumPlant())
saturation = builder.AddSystem(Saturation(min_value=[-3], max_value=[3]))
builder.Connect(saturation.get_output_port(0), pendulum.get_input_port(0))
controller = builder.AddSystem(SwingUpAndBalanceController(pendulum))
builder.Connect(pendulum.get_output_port(0), controller.get_input_port(0))
builder.Connect(controller.get_output_port(0), saturation.get_input_port(0))

# Setup visualization
scene_graph = builder.AddSystem(SceneGraph())
PendulumGeometry.AddToBuilder(
    builder, pendulum.get_state_output_port(), scene_graph
)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

logger = builder.AddSystem(VectorLogSink(2))
builder.Connect(pendulum.get_output_port(0), logger.get_input_port(0))

diagram = builder.Build()
simulator = Simulator(diagram)
context = simulator.get_mutable_context()

if running_as_notebook:
    simulator.set_target_realtime_rate(1.0)

context.SetTime(0.0)
context.SetContinuousState(
    [0, 0.1]
)

simulator.Initialize()
meshcat.StartRecording()
simulator.AdvanceTo(8)
meshcat.StopRecording()
meshcat.PublishRecording()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=711f3004-0a9b-43a3-abc8-ecd3b79bb722' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>